First execute the oneliner:

cat ~/Work/dataset/rm15/outdata/chr-loc-strand-umi_q5 | cut -f-5 | uniq -c | awk '{print $2,$3,$4,$5,$6,$1}' |tr " " ","> chr-loc-loc-strand-umi-count

In [243]:
import csv
import numpy as np
filename = 'chr-loc-loc-strand-umi-count'

with open(filename, 'rb') as f:
    reader = csv.reader(f)
    data = list(reader)

First group proximal reads which have the same UMI, by adding the sparse ones to the most frequent ones.

In [244]:
row_old = data[0]
data_aggegated_by_umi_identity = [row_old]
for row in data[1:]:
    if (row_old[0]==row[0] and row_old[3]==row[3] and row_old[4]==row[4]):
        if int(row[5]) >= int(row_old[5]):
            row[5] = str(int(row[5])+int(row_old[5]))
        else:
            row[5] = str(int(row[5])+int(row_old[5]))
            row[1:3] = row_old[1:3]
        del data_aggegated_by_umi_identity[-1]
    data_aggegated_by_umi_identity.append(row)
    row_old = row

Then group consecutive reads by proximity in location-space (max 30 bp apart) and by similarity in umi space, i.e. if there are no more that 2 mismatches.

In [245]:
row_old = data_aggegated_by_umi_identity[0]
data_aggegated_by_umi_similarity = [row_old]
space_gap = 30 
mm_gap = 2
for row in data_aggegated_by_umi_identity[1:]:
    s1 = row_old[4]
    s2 = row[4]
    numb_mismatches = sum(c1!=c2 for c1,c2 in zip(s1,s2))
    dist = abs(int(row[1])-int(row_old[1]))
    if (row_old[0]==row[0] and dist<=space_gap and row_old[3]==row[3] and numb_mismatches<=mm_gap):
        if int(row[5]) >= int(row_old[5]):
            row[5] = str(int(row[5])+int(row_old[5]))
        else:
            row[5] = str(int(row[5])+int(row_old[5]))
            row[1:3] = row_old[1:3]
            row[4] = row_old[4]
        del data_aggegated_by_umi_similarity[-1]
    data_aggegated_by_umi_similarity.append(row)
    row_old = row

In [246]:
thefile = open(filename+'_grouped', 'w') 
for item in data_aggegated_by_umi_similarity:
  print>>thefile, item

Then execute the oneliner:

cat filename_aux | tr -d "," | tr -d "'" | tr -d "[" | tr -d "]" | tr " " "\t" > filename_grouped